In [1]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

In [2]:
model = AutoModelForSequenceClassification.from_pretrained('model/fine_tuned_BERT')
tokenizer = AutoTokenizer.from_pretrained("tokenizer")

In [32]:
# helper functions

def preProcessInput(titles, summaries, genres):
# titles: list of strings in the form: [title_1, title_2, ...]
# summaries: list of summaries(strings) in the form: [summary_1, summary_2, ...]
# genres: list of genres in the form: [[genres_1], [genres_2], ...] with genres_i = "genres_i1", "genres_i2", ...
    
    inputs = []
    
    for i in range(len(titles)):
        # normalice spacing in the titles
        title_i = (' ').join(titles[i].split())
        
        # normalice spacing in the summaries
        summary_i = (' ').join(summaries[i].split())
        
        if genres[i] == []:
            genres_i = 'NonGiven'
        else:
            # convert the lists of genres to strings separated by '|'
            genres_i = '|'.join(genres[i])
            
        input_i = {'title': title_i, 'summary': summary_i, 'genres': genres_i}
        inputs.append(input_i)
        
    return inputs


def tokenizeInputs(inputs):
    
    title_mod = [movie['title'] + '<SEP>' + movie['summary'] for movie in inputs]
    genres_list = [movie['genres'] for movie in inputs]
    
        
    return tokenizer(title_mod, genres_list, padding = 'max_length', truncation = True)



    
    

In [33]:
movies = preProcessInput(["this is the    first title", "here is another title"], 
                ["the first movie   is about icecream", "the second movie is about forests"], 
                [["Action", "Fiction"], ["Romance", "Thriller"]])

In [34]:
movies

[{'title': 'this is the first title',
  'summary': 'the first movie is about icecream',
  'genres': 'Action|Fiction'},
 {'title': 'here is another title',
  'summary': 'the second movie is about forests',
  'genres': 'Romance|Thriller'}]

In [35]:
[m['genres'] for m in movies]

['Action|Fiction', 'Romance|Thriller']

In [36]:
tokenizeInputs(movies)

{'input_ids': [[101, 1142, 1110, 1103, 1148, 1641, 133, 12342, 2101, 135, 1103, 1148, 2523, 1110, 1164, 2854, 13782, 2312, 102, 6605, 197, 9713, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [16]:
model(input_1)

TypeError: list indices must be integers or slices, not tuple